In [6]:
import numpy as np
import numpy
import pyscf
import pyscf.gto
import pyscf.qmmm
import pyscf.scf
import pyscf.dft
import matplotlib.pyplot as plt
import basis_set_exchange as bse
angstrom = 1 / 0.52917721067

mol = pyscf.gto.Mole()
mol.atom = 'N 0 0 0.; N 0 0 1.'
mol.basis = {'N': bse.get_basis('6-31G', 'N', fmt='nwchem')}
mol.verbose = 0
mol.build()

deltaZ = np.array((0.05,0.05))
includeonly = [0, 1]

calc = pyscf.scf.RHF(mol)

mf = pyscf.qmmm.mm_charge(calc, mol.atom_coords()/angstrom, deltaZ)
class NoSelfQMMM(mf.__class__):
    def energy_nuc(self):
        q = mol.atom_charges().astype(np.float)
        q[includeonly] += deltaZ
        return mol.energy_nuc(q)
mf = NoSelfQMMM()

hfe=mf.kernel(verbose=0)
mycc = pyscf.cc.CCSD(mf).run()
dm1 = mycc.make_rdm1()
dm1_ao = np.einsum('pi,ij,qj->pq', mf.mo_coeff, dm1, mf.mo_coeff.conj())

# Total energy of SCF run
print ('TOTAL_ENERGY', mycc.e_tot)

# EPN from electron density
for site in includeonly:
    mol.set_rinv_orig_(mol.atom_coords()[site])
    print ('EPN', site, np.matmul(dm1_ao, mol.intor('int1e_rinv')).trace())

# Dipole
dipole = calc.dip_moment(dm=dm1, verbose=0)
print ('DIPOLE', *dipole)

# Ionic Forces


TOTAL_ENERGY -110.87228414359406
EPN 0 22.058213066869808
EPN 1 22.05821306685007
DIPOLE 0.07366988509306559 -0.006698368953782549 33.2268131443229


In [23]:
# grid comparison
def get_grid(mol):
    grid = pyscf.dft.gen_grid.Grids(mol)
    grid.level = 3
    grid.build()
    return grid.coords, grid.weights
gridcoords, gridweights = get_grid(mol)

In [24]:
ao_value = pyscf.dft.numint.eval_ao(mol, gridcoords, deriv=0)

In [25]:
rho = pyscf.dft.numint.eval_rho(mol, ao_value, dm1_ao, xctype='LDA')

In [30]:
def epn_grid(gridcoords, gridweights, rho, mol):
    for site in range(2):
        ds = np.linalg.norm(gridcoords - mol.atom_coords()[site], axis=1)
        print ((rho * gridweights/ds).sum())
epn_grid(gridcoords, gridweights, rho,mol)

22.058212704965612
22.058212704945873


In [36]:
def force_grid(gridcoords, gridweights, rho, mol):
    for site in range(2):
        rvec = gridcoords - mol.atom_coords()[site]
        print ([(rho*gridweights * rvec[:, _] / np.linalg.norm(rvec, axis=1)**3).sum() for _ in range(3)])
force_grid(gridcoords, gridweights, rho,mol)

[1.1779856604055006e-16, 1.5157146371347352e-16, 1.6506560950760947]
[-1.9125326322644298e-16, -5.314759049523943e-16, -1.6506560951706524]
